In [ ]:
# Code for creating graph data structure for muon classification such as mu+_up, mu-_up, mu+ down, mu-_down
# The csv files are generated from simulation of ICAL detector using Geant4. Refer https://github.com/Hemalatanayak/ical_g4/ for more details on detector simulation
# ical_output_mu+_up_10GeV_nt_ical_data.csv is generated by shooting a mu+ particle from top of the detector in negative z-direction having energy 10GeV
# ical_output_mu-_up_10GeV_nt_ical_data.csv is generated by shooting a mu- particle from top of the detector in negative z-direction having energy 10GeV
# ical_output_mu+_down_10GeV_nt_ical_data.csv is generated by shooting a mu+ particle from bottom of the detector in positive z-direction having energy 10GeV
# ical_output_mu-_down_10GeV_nt_ical_data.csv is generated by shooting a mu- particle from bottom of the detector in positive z-direction having energy 10GeV
# Files are imported, then concatenated, followed by digitization and then graph data structure is formed. For concatenating, digitizing, labelling and graph formation refer https://github.com/Hemalatanayak/ical_graph_data.
# Graphs are labeled as 0,1,2,3 i.e four classes are there. where 0 refers to mu+ up, 1 refers to mu- up, 2 refers to mu+ down and 3 refers to mu-_down.
# same procedure is repeated for other energies i.e 1 GeV to 10 GeV and the files are saved in zip format.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['DGLBACKEND'] = 'pytorch'
import numpy as np
from scipy.spatial.distance import cdist
import torch
!pip install dgl
import dgl
import shutil


# from google.colab import drive
# drive.mount('/content/drive')
# import the needed files
df1=pd.read_csv("...filePath/ical_output_mu+_up_10GeV_nt_ical_data.csv",skiprows=12,sep=",",names=["event_id","x","y","z","t","Px","Py","Pz"])#mu+ up
df2=pd.read_csv('...filePath/ical_output_mu-_up_10GeV_nt_ical_data.csv',skiprows=12,sep=",",names=["event_id","x","y","z","t","Px","Py","Pz"])#mu- up
df3=pd.read_csv('...filePath/ical_output_mu+_down/ical_output_mu+_down_10GeV_nt_ical_data.csv',skiprows=12,sep=",",names=["event_id","x","y","z","t","Px","Py","Pz"])#mu+ down
df4=pd.read_csv('...filePath/ical_output_mu-_down/ical_output_mu-_down_10GeV_nt_ical_data.csv',skiprows=12,sep=",",names=["event_id","x","y","z","t","Px","Py","Pz"])#mu- down

df=[df1,df2,df3,df4]
muon_data=pd.concat(df) #concatenated data
print(type(muon_data))
print(muon_data)

#digitize the concatenated data 
# the size of the detector along x/y is 16 m. There are 512 strips in each direction
p=np.arange(-8000,8000,1) #X,Y Coordinate
q=np.arange(0,512,1)      #(Strip no.)
m=max(q)/np.size(p)
# the size of the detector along z is 14.4 m. There are 150 layers (detectors).
r=np.arange(-7180,7280,1)  #Z- Coordinate 
layer=np.arange(1,151,1)  #layer numbers (150 layers)
slope=max(layer)/np.size(r)

event_id=muon_data['event_id']
x_dig=np.round((m*muon_data['x'])+255.5)
y_dig=np.round((m*muon_data['y'])+255.5)
z_dig=np.round((slope*muon_data['z'])+75.5)
t=muon_data['t']
Px=muon_data['Px']
Py=muon_data['Py']
Pz=muon_data['Pz']
digitized_data=pd.DataFrame({'event_id':event_id,'x_dig':x_dig,'y_dig':y_dig,'z_dig':z_dig,'t':t,'px':Px,'py':Py,'pz':Pz })


#Label the concatenated data
ranges = [(0, len(df1)), (len(df1), len(df2)+len(df1)), (len(df2)+len(df1), len(df3)+len(df2)+len(df1)),(len(df3)+len(df2)+len(df1), len(df4)+len(df3)+len(df2)+len(df1))]
digitized_data['graphLabel'] = 0
for i, (start, end) in enumerate(ranges):
    digitized_data['graphLabel'][start:end] = i

# The format of graphID is a combination of graphlabel and eventID.
digitized_data = digitized_data.reset_index(drop=True)
digitized_data['graphID'] = 0
for i, (start, end) in enumerate(ranges):
    digitized_data.loc[start:end-1, 'graphID'] = (i)*10000 + digitized_data.loc[start:end-1, 'event_id']

#Create nodeID
data = []
for x, y, z in zip(digitized_data['x_dig'], digitized_data['y_dig'], digitized_data['z_dig']):
    nodeID = z + (y * 150) + (x * 150 * 512)
    data.append(nodeID)
digitized_data['nodeID'] = data
digitized_data['Energy'] = 10   #Give the corresponding energy]
print((digitized_data))

#Create graph data structure
if not os.path.exists('ical_muon_classification_graphs_10'):
    os.makedirs('ical_muon_classification_graphs_10')

# Loop over all unique graph IDs in the DataFrame and plotting graph for each graphID
for graphID in digitized_data['graphID'].unique():
    graph_df = digitized_data[digitized_data['graphID'] == graphID][['x_dig', 'y_dig', 'z_dig', 't','graphLabel','graphID','nodeID','Energy']]
    num_nodes = len(graph_df)
    if num_nodes > 9:
        g = dgl.graph((np.arange(0,num_nodes-1), np.arange(1,num_nodes)))
        g.ndata['pos']=torch.tensor(graph_df[['x_dig','y_dig','z_dig']].values)
        t = torch.tensor(graph_df['t'].to_numpy())
        g.edata["t_diff"]=t[1:]-t[:-1]
        Energy = graph_df['Energy'].iloc[0]

        dgl.save_graphs(f"ical_muon_classification_graphs_10/graph_{graphID}_{Energy}.dgl", [g],{'node_feat': g.ndata['pos'], 'edge_feat': g.edata['t_diff']})
# folder_path = "/content/ical_muon_classification_graphs_5"
# num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
# print("Number of files in folder:", num_files)

folder_name = 'ical_muon_classification_graphs_10'
shutil.make_archive("ical_muon_classification_graphs_10", 'zip', folder_name)

